In [4]:
import pandas as pd
import datetime
from tqdm.notebook import tqdm
import json
from collections import defaultdict
from matplotlib import pyplot as plt
import numpy as np

In [5]:
!ls ../usc-x-24-us-election/

README.md  part_12  part_16  part_2   part_23  part_27	part_5	part_9
part_1	   part_13  part_17  part_20  part_24  part_28	part_6
part_10    part_14  part_18  part_21  part_25  part_3	part_7
part_11    part_15  part_19  part_22  part_26  part_4	part_8


In [6]:
import os
from glob import glob

In [7]:
def parts_sort_key(key):
    if "may_july" in key:
        return 0
    return 1
dataset_parts = sorted(glob("../usc-x-24-us-election/part*/*csv.gz"), key=lambda x: (parts_sort_key(x), int(x[:-len(".csv.gz")].split("_")[-1])))

In [19]:
!ls -alh ../usc-x-24-us-election/part_1/may_july_chunk_5.csv.gz

-rw-rw-r-- 1 ubuntu ubuntu 19M Nov 17 15:59 ../usc-x-24-us-election/part_1/may_july_chunk_5.csv.gz


In [8]:
dataset_parts

['../usc-x-24-us-election/part_1/may_july_chunk_1.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_2.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_3.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_4.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_5.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_6.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_7.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_8.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_9.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_10.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_11.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_12.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_13.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_14.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_15.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_16.csv.gz',
 '../usc-x-24-us-election/part_1/may_july_chunk_1

In [9]:
len(dataset_parts)

557

In [11]:
def find_common_columns(common = None, _print=False):
    for df_p in tqdm(dataset_parts):
        df = pd.read_csv(df_p, compression='gzip', dtype=str, nrows=5)
        cols = set(df.columns)
        if common is None:
            common = cols
            continue
        if cols - common or common - cols:
            if _print:
                print((cols - common).union(common - cols))
        common &= cols
    return common

In [12]:
common_cols = find_common_columns(find_common_columns(), _print=False)

  0%|          | 0/557 [00:00<?, ?it/s]

  0%|          | 0/557 [00:00<?, ?it/s]

In [13]:
df = pd.read_csv(dataset_parts[-150], compression='gzip', header=0, dtype=str, usecols=common_cols)
df = df[~df['epoch'].isna()]
df = df[~(df['epoch'].str.contains('e+'))]
df['epoch'] = pd.to_datetime(df['epoch'].astype(float), unit='s')

In [14]:
df.shape

(50000, 28)

In [15]:
len(dataset_parts)*50000

27850000

In [9]:
# let's print the data
for key, value in df[df.hashtags != '[]'].iloc[0].to_dict().items():
    print("#### column:", key)
    print(value)

#### column: id
1796668866463990187
#### column: text
@JAHeale @ElectCalculus @findoutnow Wow 7 more seats &amp; they will be the third party! 
#VoteTactically https://t.co/CFximIdoTI
#### column: url
https://twitter.com/FaBPaule/status/1796668866463990187
#### column: epoch
2024-05-31 22:23:32
#### column: media
PW
#### column: retweetedTweet
False
#### column: retweetedTweetID
nan
#### column: retweetedUserID
nan
#### column: id_str
1796668866463990187
#### column: lang
en
#### column: rawContent
@JAHeale @ElectCalculus @findoutnow Wow 7 more seats &amp; they will be the third party! 
#VoteTactically https://t.co/CFximIdoTI
#### column: replyCount
0
#### column: retweetCount
1.0
#### column: likeCount
5.0
#### column: quoteCount
0.0
#### column: conversationId
1.7966345552327808e+18
#### column: conversationIdStr
1796634555232780702
#### column: hashtags
[{'indices': [90, 105], 'text': 'VoteTactically'}]
#### column: mentionedUsers
[{'id_str': '775680607', 'name': 'James Heale', 'scr

In [10]:
def parse_view_count(data):
    try:
        data = eval(data)
        return data['count']
    except Exception:
        return 0

In [11]:
USER_COLS = [
     'created',
     'followersCount',
     'friendsCount',
     'statusesCount',
     'verified',
     'blue',
]
def parse_user(data):
    import datetime
    try:
        data = eval(data)
        return tuple(data[key]
             for key in USER_COLS)
    except Exception as e:
        # print(e)
        return tuple(np.nan for key in USER_COLS)

In [12]:
COLUMNS_TO_LOAD = [
    'user',
    'url',
    'text',
    'replyCount',
    'retweetCount',
    'likeCount',
    'quoteCount',
    'viewCount',
    'epoch',
    'lang'
]
def load_df(i, pth):
    df = pd.read_csv(pth, compression='gzip', header=0, dtype=str, usecols=COLUMNS_TO_LOAD)
    # only english
    df = df[df['lang'] == 'en']
    df.drop(labels=['lang'], axis=1, inplace=True)
    # keep the file index
    # df['df_i'] = i
    
    df['url'] = df['url'].str.split('/').str[-1]
    df = df[~df['url'].isna()]
    df = df[(df['url'].str.len() > 3) & (df['url'] != 'False') & (df['url'].str.isnumeric())]

    # dt cleaning
    df = df[~df['epoch'].isna()]
    df = df[~df['epoch'].str.contains('e+')]
    df['epoch'] = pd.to_datetime(pd.to_numeric(df['epoch'], errors='coerce'), unit='s')
    df = df[~df['epoch'].isna()]
    df = df[df.epoch != 'PW']

    # view count
    df['viewCount'] = df['viewCount'].apply(parse_view_count)

    # other stats columns
    for col in ['retweetCount', 'likeCount', 'quoteCount', 'replyCount']:
        df[col] = df[col].str.split('.').str[0].astype(int)

    # user columns
    df['user_created'], df['user_followers'], df['user_following'], df['user_tweets'], df['user_verified'], df['user_blue'] = zip(*df['user'].apply(parse_user))

    df.rename(columns={'epoch': 'dt', 'url': 'tweet_id'}, inplace=True)
    df.drop(labels=['user', ], axis=1, inplace=True)
    return df

In [13]:
from joblib import Parallel, delayed

In [14]:
result = Parallel(n_jobs=7)(delayed(load_df)(i, x) for i, x in enumerate(tqdm(dataset_parts)))

  0%|          | 0/557 [00:00<?, ?it/s]

In [15]:
big_df = pd.concat(result)
big_df.drop_duplicates(subset='tweet_id', keep='last', inplace=True)
big_df.drop(columns=['tweet_id'], inplace=True)

In [16]:
big_df.shape

(23695429, 13)

In [17]:
big_df['dt'].min(), big_df['dt'].max()

(Timestamp('2009-10-13 19:15:12'), Timestamp('2024-11-06 15:46:22'))

In [21]:
big_df.to_csv('data.csv', index=False)

In [22]:
(big_df.dt >= np.datetime64('2024-10-01')).sum()

np.int64(932159)

In [24]:
small_df = big_df[(big_df.dt >= np.datetime64('2024-10-01'))].copy()
small_df.sort_values(by='dt').to_csv('data_small.csv', index=False)